# Task Two: Flow, Density, and Speed Contours

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Define file path

In [65]:
# Group 2 US101SB
stations_us101_g2_path = "Group 2 US101SB/Stations_US101.csv"
zone_readings_10_19_path = "Group 2 US101SB/Zone_Readings_10-19.csv"
zone_readings_10_26_path = "Group 2 US101SB/Zone_Readings_10-26.csv"

# SR_91_Bottleneck_B_150-200
inventory_sr91_b_path = "SR_91_Bottleneck_B_150-200/Inventory.csv"
zone_readings_0831_0901_path = "SR_91_Bottleneck_B_150-200/Zone_Readings_0831-0901.csv"
zone_readings_0907_0908_path = "SR_91_Bottleneck_B_150-200/Zone_Readings_0907-0908.csv"

## 2. Load data

In [66]:
# Read all zoning readings data
zone_readings_10_19 = pd.read_csv(zone_readings_10_19_path)
zone_readings_10_26 = pd.read_csv(zone_readings_10_26_path)
# zone_readings_0831_0901 = pd.read_csv(zone_readings_0831_0901_path)
# zone_readings_0907_0908 = pd.read_csv(zone_readings_0907_0908_path)

## 3. Explore data

In [67]:
# Drop the index
zone_readings_10_19 = zone_readings_10_19.drop(columns=['index'])
zone_readings_10_19.head()

,date,station_ID,volume,speed,occupancy
0,2019-10-19 00:00:00,402005,0.0,0.000,0.000000
1,2019-10-19 00:00:00,405389,0.0,0.000,0.000000
2,2019-10-19 00:00:00,405390,8.0,75.375,0.012800
3,2019-10-19 00:00:00,405394,0.0,0.000,0.000000
4,2019-10-19 00:00:00,405395,2.0,69.000,0.004467


In [68]:
# Calculate the density = (52.5 / (l_v * l_D)) * occupancy
lv = 18 # ft
l_D = 6 # ft
zone_readings_10_19['density'] = (52.5 / (lv * l_D)) * zone_readings_10_19['occupancy']
zone_readings_10_26['density'] = (52.5 / (lv * l_D)) * zone_readings_10_26['occupancy']
zone_readings_10_19.head()

,date,station_ID,volume,speed,occupancy,density
0,2019-10-19 00:00:00,402005,0.0,0.000,0.000000,0.000000
1,2019-10-19 00:00:00,405389,0.0,0.000,0.000000,0.000000
2,2019-10-19 00:00:00,405390,8.0,75.375,0.012800,0.006222
3,2019-10-19 00:00:00,405394,0.0,0.000,0.000000,0.000000
4,2019-10-19 00:00:00,405395,2.0,69.000,0.004467,0.002171


In [69]:
# Select station_ID = '405390'
station = zone_readings_10_19.loc[zone_readings_10_19['station_ID'] == 405390]
station.head()

,date,station_ID,volume,speed,occupancy,density
2,2019-10-19 00:00:00,405390,8.0,75.375000,0.012800,0.006222
147,2019-10-19 00:00:00,405390,3.0,69.666667,0.004750,0.002309
292,2019-10-19 00:01:00,405390,3.0,62.666667,0.005025,0.002443
437,2019-10-19 00:01:00,405390,8.0,68.125000,0.013075,0.006356
582,2019-10-19 00:02:00,405390,4.0,58.500000,0.012650,0.006149


## 4. Define function to create contour

In [70]:
# Genrate countour of given metrics, the metrics are volume, speed, density.
# Sort data by station_ID, time.
# Get metric value of each station_ID.
# Create a new contour_df, row is date, column is station_ID, value is metric value.
def create_contour(df, metric):
    df = df.sort_values(by=['station_ID', 'date']).reset_index(drop=True)
    station_IDs = df['station_ID'].unique()
    for station_ID in station_IDs:
        station = df.loc[df['station_ID'] == station_ID]
        station = station.reset_index(drop=True)
        if station_ID == station_IDs[0]:
            contour_df = pd.DataFrame({'date': station['date'], station_ID: station[metric]})
        else:
            station_metric = pd.DataFrame({station_ID: station[metric]})
            contour_df = pd.concat([contour_df, station_metric], axis=1)

    return contour_df

metrics = ['volume', 'speed', 'density']
contour = create_contour(zone_readings_10_19, metrics[1])
contour.head()

,date,400000,400011,400143,400304,400382,400392,400460,400513,400525,...,410553,412439,412443,412446,415697,415704,416620,416911,418244,421965
0,2019-10-19 00:00:00,0.0,71.222222,66.750000,48.500000,84.571429,60.0,65.142857,80.25,69.714286,...,67.70,65.000000,67.333333,82.666667,0.0,0.0,41.200000,93.0,72.000000,0.0
1,2019-10-19 00:00:00,0.0,79.750000,73.800000,71.250000,67.625000,64.4,62.000000,75.00,65.500000,...,71.00,56.000000,60.000000,81.000000,0.0,0.0,81.000000,89.0,66.090909,0.0
2,2019-10-19 00:01:00,0.0,62.500000,85.777778,73.125000,73.727273,69.5,69.333333,80.00,53.666667,...,68.25,73.500000,71.000000,82.666667,0.0,0.0,44.500000,81.0,0.000000,0.0
3,2019-10-19 00:01:00,0.0,57.428571,78.250000,76.833333,80.600000,74.5,71.000000,89.50,69.333333,...,65.00,67.000000,64.444444,94.600000,0.0,0.0,83.428571,64.0,66.142857,63.0
4,2019-10-19 00:02:00,0.0,65.000000,72.000000,0.000000,49.333333,78.0,75.666667,80.00,65.000000,...,0.00,61.666667,78.000000,82.600000,0.0,0.0,82.500000,89.0,65.500000,67.0


## 5. Process the data

In [71]:
metrics = ['volume', 'speed', 'density']

# Create contour file for US101SB 10-19
for metric in metrics:
    contour_df = create_contour(zone_readings_10_19, metric)
    contour_csv_path = f"Group 2 US101SB/{metric}_contour_10-19.csv"
    contour_df.to_csv(contour_csv_path, index=False)

# Create contour file for US101SB 10-26
for metric in metrics:
    contour_df = create_contour(zone_readings_10_26, metric)
    contour_csv_path = f"Group 2 US101SB/{metric}_contour_10-26.csv"
    contour_df.to_csv(contour_csv_path, index=False)
